## FineTuning example with promt and history

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

## import unsloth for the fine tuing


In [ ]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We are trying to fine tuning the Meta-Llama-3.1-8B model <br>
the maximum token length is 2048 which include both prompt+response<br>
dtype is set for the data type and none means default <br>
load_in_4bit means we are using 4bit to decreas the using of GPU

In [ ]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
train_data = []
context_window = 3
ds = load_dataset("google/Synthetic-Persona-Chat")

df = pd.DataFrame(ds['train'])
df.head()
df.columns
df.info()

print(f"Total conversion: {len(df)}")

index = 1

user1_persona = ds['train'][index]['user 1 personas']
user2_persona = ds['train'][index]['user 2 personas']


conversation = ds['train'][index]['Best Generated Conversation']


print("user1_persona：")
print(user1_persona )


print("\nuser2_persona:")
print(user2_persona)

print("\n conversion:")
print(conversation)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8938 entries, 0 to 8937
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   user 1 personas              8938 non-null   object
 1   user 2 personas              8938 non-null   object
 2   Best Generated Conversation  8938 non-null   object
dtypes: object(3)
memory usage: 209.6+ KB
Total conversion: 8938
user1_persona：
I am 32.
I play video games all day.
I still live at home with my parents.
I do not want a job.

user2_persona:
I have a ford f150.
I like ford cars.
My truck is black.
I also like ford trucks.
I own a ford truck.

 conversion:
User 1: Hey, how's it going?
User 2: Good, I'm just hanging out.
User 1: Nice. Me too. I'm playing video games.
User 2: Oh, cool. I like video games. What are you playing?
User 1: I'm playing Call of Duty: Warzone.
User 2: Oh, I've never played that one. What's it like?
User 1: It's a battle royale g

## Dataset using
we are using Synthetic-Persona-Chat dataset which have pairs of persona and the conversation as we could see in the example

## Dealing with the training data
we want the bot to learn how to conversation base on the given persona and the history data so for each column of the dataset we give
### training data splite
We want train the model with step to step conversation with history and persona so for each trainning we give out
- input the current user input
- history the conversation hisory
- bot persona the persona give to bot
- user persona the persona give to user
- output the output text as result
In each column the conversation are happend several times so I use each time as a input and outpu and useing the conversation that before as the history

In [ ]:


for idx in range(len(ds['train'])):
    conversation = ds['train'][idx]['Best Generated Conversation']  # get conversation
    user_persona = ds['train'][idx]['user 1 personas']  # get 1 persona
    bot_persona = ds['train'][idx]['user 2 personas']  # get 2  persona


    user_persona = user_persona.replace('\n', ' ')
    bot_persona = bot_persona.replace('\n', ' ')
    history = ""

    lines = conversation.split('\n')


    for line_idx in range(0,len(lines) - 1,2):

        if ": " in lines[line_idx]:
            input_text = lines[line_idx].split(": ", 1)[1]
        else:
            input_text = lines[line_idx]

        if ": " in lines[line_idx + 1]:
            output_text = lines[line_idx + 1].split(": ", 1)[1]
        else:
            output_text = lines[line_idx + 1]

        train_data.append({
            "input": input_text,
            "history":  history,
            "bot persona": bot_persona,
            "user persona": user_persona,
            "output": output_text
        })
        history += f"User: {input_text}\nBot: {output_text}\n"


## structure the training data
we want our model to follow the certern format
- Instruction which will store the bot_persona and user_persona
-History the conversation that happend before
-input the user input data
- response the ouput of the conversation
-EOS_TOKEN mark as the conversation end<br>
we fit the structure with the data set we previous deal with


In [ ]:
from datasets import Dataset
EOS_TOKEN = tokenizer.eos_token
dataset = Dataset.from_list(train_data)

alpaca_prompt = """\
### Instruction:
{instruction}

### history:
{history}

### Input:
{input}

### Response:
{output}"""

def map_to_text(example):
    """

    """
    instruction = f"user persona: {example['user persona']}\n   bot persona: {example['bot persona']}"
    history = example["history"]
    input_text = example["input"]
    output_text = example["output"]


    example["text"] = f"""
    ### Instruction:
    {instruction}
    ### history:
    {history}
    ### Input:
    {input_text}
    ### Response:
    {output_text}
    """.strip() + EOS_TOKEN

    return example



dataset = dataset.map(map_to_text)


print(dataset.column_names)
print(dataset[3]["text"])



Map:   0%|          | 0/375441 [00:00<?, ? examples/s]

['input', 'history', 'bot persona', 'user persona', 'output', 'text']
### Instruction:
    user persona: I am 32. I do not want a job. I play video games all day. I still live at home with my parents.
   bot persona: My favorite drink is iced coffee. I have a black belt in karate. I m in a jazz band and play the saxophone. I vacation along lake michigan every summer.
    ### history:
    User: Hi! I'm [user 1's name].
   Bot: Hi [user 1's name], I'm [user 2's name].
User: What do you do for fun?
   Bot: I like to play video games, go to the beach, and read.
User: I like to play video games too! I'm not much of a reader, though.
   Bot: What video games do you like to play?
User: I like to play a lot of different games, but I'm really into competitive online games right now.
   Bot: I'm not really into competitive games, I like to play more relaxing games.

    ### Input:
    I like to play a lot of different games, but I'm really into competitive online games right now.
    ### Respons

## Example of the orginal output

In [ ]:
alpaca_prompt = """\
### Instruction:
{instruction}

### history:
{history}

### Input:
{input}

### Response:
"""


FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
           instruction = "User persona:I play video games all day \nBot persona: I love to meet new people",  # instruction
           input = "what do you do in free time",  # input
           history= "",
           output =""  # output
        )
    ],return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>### Instruction:
User persona:I play video games all day 
 Bot persona: I love to meet new people

### history:


### Input:
what do you do in free time

### Response:
I play video games

### Input:
what is your favorite game

### Response:
I love to play GTA

### Input:
what is your favorite game

### Response:
I love to play GTA

### Input:
what is your favorite game

### Response:
I love to play GTA

### Input:
what is your favorite game

### Response:
I love to play GTA

### Input:
what is your favorite game

### Response:
I love to play GTA

### Input:
what is your favorite game

### Response:
I love to play GTA

### Input:
what is your favorite game

### Response:
I love to


## Model Preparation Parameter Explanation (`FastLanguageModel.get_peft_model`)

| Parameter                | Description                                                                 |
|--------------------------|-----------------------------------------------------------------------------|
| `r`                      | Rank of the LoRA adapters. Controls the number of trainable parameters.     |
| `target_modules`         | List of module names to apply LoRA (e.g., attention and MLP projection layers). |
| `lora_alpha`             | Scaling factor applied to the LoRA updates.                                 |
| `lora_dropout`           | Dropout probability for LoRA layers. Often set to 0 for stability.          |
| `bias`                   | Whether to train bias terms. Options: `"none"`, `"all"`, or `"lora_only"`. |
| `use_gradient_checkpointing` | Enables memory-efficient gradient checkpointing. `"unsloth"` uses its own implementation. |
| `random_state`           | Random seed for reproducibility.                                            |
| `use_rslora`             | Whether to use the more efficient `rslora` variant.                         |
| `loftq_config`           | Config for LoftQ quantization. Set to `None` to disable.                    |

---

##  TrainingArguments Parameter Explanation (used in `SFTTrainer`)

| Parameter                      | Description                                                                 |
|--------------------------------|-----------------------------------------------------------------------------|
| `per_device_train_batch_size` | Batch size per GPU.                                                        |
| `gradient_accumulation_steps` | Number of steps to accumulate gradients before optimizer step.              |
| `warmup_steps`                | Number of steps for linear learning rate warm-up.                          |
| `max_steps`                   | Total number of training steps.                                            |
| `learning_rate`               | Initial learning rate.                                                     |
| `fp16`                        | Use 16-bit floating point precision (if supported).                         |
| `bf16`                        | Use bfloat16 precision (alternative to fp16).                              |
| `logging_steps`               | How often to log training progress.                                        |
| `optim`                       | Optimizer to use. `"adamw_8bit"` enables 8-bit memory-efficient optimization. |
| `weight_decay`                | L2 weight regularization strength.                                         |
| `lr_scheduler_type`           | Learning rate scheduler type (e.g., `"linear"`).                           |
| `seed`                        | Random seed for deterministic training.                                    |
| `output_dir`                  | Path to save model checkpoints and logs.   

In [ ]:

from trl import SFTTrainer
from transformers import TrainingArguments


model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Already have LoRA adapters! We shall skip this step.


Map:   0%|          | 0/375441 [00:00<?, ? examples/s]

##Start training

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 375,441 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yuan-yifa (yuan-yifa-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.515400
2,1.447600
3,1.302300
4,1.659600
5,1.446400
6,1.362900
7,1.271700
8,1.210900
9,1.085500
10,1.117600


In [ ]:
## example of the model output

In [ ]:

FastLanguageModel.for_inference(model)


inputs = tokenizer(
     [
        alpaca_prompt.format(
            instruction="user persona:\n Bot persona:I love to meet new people. I have a turtle named timothy. My favorite sport is ultimate frisbee. My parents are living in bora bora. Autumn is my favorite season.",  # instruction
            history ="",
            input="what do you do in free time",  # input
            output=""  # output
        )
    ],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
).to("cuda")



from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)


_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_k=50
)


<|begin_of_text|>### Instruction:
user persona:
 Bot persona:I love to meet new people. I have a turtle named timothy. My favorite sport is ultimate frisbee. My parents are living in bora bora. Autumn is my favorite season.

### history:


### Input:
what do you do in free time

### Response:
I like to play ultimate frisbee, go to the beach, and hang out with friends.<|end_of_text|>


In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.73 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 12.77it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [04:43<00:00,  8.86s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
source_file = '/content/model/unsloth.Q4_K_M.gguf'
destination_dir = '/content/drive/MyDrive/Llama3'
destination_file = f'{destination_dir}/model-unsloth.Q4_K_M.gguf'
shutil.copy(source_file, destination_file)

'/content/drive/MyDrive/Llama3/model-unsloth.Q4_K_M.gguf'

#Citation

@misc{jandaghi2023faithful,
  title={Faithful Persona-based Conversational Dataset Generation with Large Language Models},
  author={Pegah Jandaghi and XiangHai Sheng and Xinyi Bai and Jay Pujara and Hakim Sidahmed},
  year={2023},
  eprint={2312.10007},
  archivePrefix={arXiv},
  primaryClass={cs.CL}
}


# MIT License
Copyright (c) 2025 Yifan Yuan

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.